#### Package Imports

In [5]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import plotly.graph_objs as go
from scipy.integrate import quad
from scipy.spatial.distance import cdist
from scipy.optimize import minimize, differential_evolution, NonlinearConstraint
from sklearn.decomposition import PCA
import math
import time
import sobol_seq
import torch
import gpytorch
import copy
import rdkit
import os

from datetime import datetime, timedelta
import time

#### Objective Function

In [3]:
values = [
    0.00, 0.00, 0.00, 0.98, 0.00, 0.00, 0.00, 0.00, 0.00, 0.71,
    0.00, 0.00, 0.00, 0.00, 0.00, 1.04, 0.00, 0.00, 0.00, 0.00,
    0.00, 1.17, 0.00, 0.00, 0.00, 0.00, 0.00, 2.73, 0.00, 0.00,
    1.63, 2.99, 2.85, 2.53, 1.68, 2.10, 0.00, 0.00, 0.00, 0.96,
    0.00, 7.53, 0.00, 0.00, 0.00, 0.93, 0.00, 3.44, 0.00, 0.00,
    0.00, 1.00, 0.00, 1.25, 0.00, 0.00, 0.00, 0.00, 0.00, 2.84,
    0.00, 0.00, 0.00, 0.40, 0.00, 3.08, 0.00, 1.21, 1.33, 2.08,
    4.11, 39.52, 14.51, 91.50, 75.68, 12.76, 94.85, 100.00, 13.29,
    79.30, 56.99, 9.84, 50.66, 85.40, 21.31, 72.83, 48.16, 21.30,
    48.22, 58.76, 7.60, 53.00, 25.89, 9.62, 11.46, 96.19
]

values2 = [
    0.00, 0.00, 0.00, 0.00, 0.00, 2.85, 0.00, 0.00, 0.00, 0.00,
    0.00, 4.22, 0.00, 2.14, 3.82, 2.24, 6.73, 2.20, 0.00, 3.39,
    6.02, 4.17, 12.20, 2.17, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 1.45, 0.70, 0.00, 9.46, 14.17, 10.84,
    13.16, 26.57, 0.00, 1.23, 2.27, 2.17, 4.08, 5.85, 0.00, 2.70,
    3.22, 3.23, 13.11, 100.00, 0.89, 1.85, 2.84, 2.23, 13.15,
    66.58, 0.00, 3.00, 3.29, 1.98, 5.78, 6.53, 5.55, 59.05, 66.65,
    12.67, 71.33, 17.61, 72.85, 79.82, 77.47, 12.01, 80.70, 69.99,
    56.73, 64.38, 59.94, 7.83, 22.39, 41.40, 49.73, 63.99, 59.58,
    14.03, 25.01, 28.83, 76.45, 84.19, 80.28, 16.10, 32.63, 57.77
]

values3 = [
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 1.57, 2.47, 1.78, 2.44, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    1.08, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 1.71, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 8.24, 9.64, 7.41,
    14.62, 1.12, 80.17, 88.63, 84.63, 11.77, 90.38, 92.19, 76.17,
    90.86, 95.08, 9.87, 23.89, 77.24, 74.23, 82.87, 77.75, 16.29,
    42.93, 61.42, 79.25, 91.14, 100.00, 15.72, 27.82, 81.91
]

values4 = [
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 16.69, 38.23, 38.47, 17.68, 33.10, 59.10, 41.87,
    59.29, 56.15, 12.52, 59.85, 100.00, 29.14, 42.77, 36.08, 17.73,
    32.18, 77.39, 45.33, 50.12, 51.52, 17.36, 37.45, 81.79
]

values5 = [
    3.91, 3.88, 2.93, 2.78, 2.77, 0.00, 4.31, 2.52, 2.57, 2.26,
    2.18, 4.07, 0.00, 6.07, 6.48, 5.16, 7.71, 1.03, 0.00, 0.00,
    0.00, 0.00, 0.00, 1.22, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
    0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.27, 2.80, 3.49, 3.09,
    2.06, 2.28, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 2.58, 4.18,
    4.09, 3.12, 16.27, 11.37, 1.57, 1.52, 2.58, 1.80, 10.55, 28.43,
    0.00, 1.03, 1.07, 1.01, 1.93, 14.39, 4.94, 76.39, 77.11, 21.85,
    89.61, 83.31, 100.00, 70.56, 67.26, 20.98, 63.48, 77.69, 88.52,
    95.52, 91.53, 18.80, 61.39, 42.12, 37.24, 89.43, 81.77, 29.05,
    51.52, 27.04, 94.88, 99.53, 97.54, 31.54, 49.99, 45.94
]


#### Processing (Intermediate)

In [15]:
parent_submission_filename = 'submissions'
repeat_numbers = 1
exp_time = 0

py_files = [f for f in os.listdir(parent_submission_filename ) if f.endswith(".py")]
for py_file in py_files:
    minimumY = []
    for reaction_yield in [values,values2,values3,values4,values5]:
        
        def experiment_yield(condition , X_searchspace):
            return(-reaction_yield[X_searchspace.index(condition)])
            
        for repeat in range(repeat_numbers):
            with open(f'{parent_submission_filename }/{py_file}', "r") as f:
                script_text = f.read()
                exec(script_text, globals())
                minimumY += [min(BO_m.Y)]
                #print(BO_m.Y)
                
    print(py_file, np.average(minimumY)) 

<string>:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
<string>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


Yong_final.py -84.806


#### Processing (Advanced)

In [17]:
parent_submission_filename = 'submissions'
repeat_numbers = 1
exp_time = 0

py_files = [f for f in os.listdir(parent_submission_filename ) if f.endswith(".py")]
for py_file in py_files:
    minimumY = []
    runtime = [] 
    for reaction_yield in [values,values2,values3,values4,values5]:
        
        def experiment_yield(condition , X_searchspace):
            return(-reaction_yield[X_searchspace.index(condition)])
            
        for repeat in range(repeat_numbers):
            with open(f'{parent_submission_filename }/{py_file}', "r") as f:
                script_text = f.read()
                time1 = time.time()
                exec(script_text, globals())
                time2 = time.time()
                minimumY += [min(BO_m.Y)]
                runtime += [time2-time1]
                #print(BO_m.Y)
                
    print(py_file, np.average(minimumY), np.average(minimumY)+np.average(runtime)) 


<string>:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
<string>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


Yong_final.py -79.016 -78.5670856628418
